# Imports

In [ ]:
import asyncio
import datetime
import json
import os

import lancedb
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry
import tiktoken
import torch
import torch.nn.functional as F
from datasets import load_dataset
from dotenv import load_dotenv
from loguru import logger
from openai import OpenAI, AsyncOpenAI
from sentence_transformers import SentenceTransformer
from tenacity import (
    RetryCallState,
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

try:
    from google.colab import userdata  # type: ignore
except ImportError:
    userdata = None

load_dotenv(override=True)

True

# Preparing dataset

In [2]:
ds = load_dataset("AiresPucrs/tmdb-5000-movies", split="train")
ds

Dataset({
    features: ['id', 'budget', 'genres', 'homepage', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'cast', 'crew'],
    num_rows: 4803
})

In [3]:
# Remove missing overview
ds = ds.filter(lambda x: bool(x["overview"]))
ds

Dataset({
    features: ['id', 'budget', 'genres', 'homepage', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'cast', 'crew'],
    num_rows: 4800
})

In [4]:
def preprocess(example: dict) -> dict:
    example["genres"] = [g["name"] for g in json.loads(example["genres"])]
    example["keywords"] = [k["name"] for k in json.loads(example["keywords"])]
    example["production_companies"] = [
        pc["name"] for pc in json.loads(example["production_companies"])
    ]
    example["production_countries"] = [
        pc["name"] for pc in json.loads(example["production_countries"])
    ]
    example["release_date"] = datetime.datetime.strptime(
        d if (d := example["release_date"]) else "1970-01-01", "%Y-%m-%d"
    ).date()
    example["release_year"] = example["release_date"].year
    example["spoken_languages"] = [
        sl["name"] for sl in json.loads(example["spoken_languages"])
    ]
    example["cast"] = [
        {
            "name": c["name"],
            "character": c["character"],
        }
        for c in json.loads(example["cast"])
    ]
    example["crew"] = [
        {
            "name": c["name"],
            "job": c["job"],
        }
        for c in json.loads(example["crew"])
    ]
    return example


In [5]:
ds = ds.map(preprocess, remove_columns=["homepage"], num_proc=4)

# Embeddings

In [6]:
queries = [
    "I want to watch an exciting superhero movie",
    "我想看一部超級英雄電影",
]

movies = [
    "A movie about a group of friends who go on a road trip",
    "A romantic comedy about a couple who meet at a wedding",
    "An autobiography of George Washington, the first president of the United States",
    "Spider-Man is fighting against the Green Goblin in another universe",
]

## sentence-transformers

* [Official Documentation](https://sbert.net/)
* Models supporting `sentence-transformers`: https://huggingface.co/models?library=sentence-transformers
* Models for `sentence-similarity`: https://huggingface.co/models?pipeline_tag=sentence-similarity


In [7]:
model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", device="cpu"
)

### Compute embeddings

In [8]:
query_embeddings = model.encode(queries)
movie_embeddings = model.encode(movies)
print(query_embeddings.shape, movie_embeddings.shape)
print(f"Query: {query_embeddings[0][:5]}")
print(f"Movie: {movie_embeddings[0][:5]}")

(2, 384) (4, 384)
Query: [-0.4084644  -0.24543129 -0.600559   -0.11258549  0.19218642]
Movie: [ 0.30984214 -0.12824544 -0.33090717 -0.2385715   0.33415732]


### Computing similarity + retrieving top k

In [9]:
# Compute cosine similarities
similarities = model.similarity(query_embeddings, movie_embeddings)
similarities

tensor([[ 0.2611,  0.1753, -0.0321,  0.3556],
        [ 0.2784,  0.1435,  0.0242,  0.3834]])

In [12]:
YELLOW = "\033[33m"
END = "\033[0m"
# Output the pairs with their score
for idx_i, sentence1 in enumerate(queries):
    print(sentence1)
    for idx_j, sentence2 in enumerate(movies):
        print(f" - {sentence2: <30}: {YELLOW}{similarities[idx_i][idx_j]:.4f}{END}")

I want to watch an exciting superhero movie
 - A movie about a group of friends who go on a road trip: 0.2611
 - A romantic comedy about a couple who meet at a wedding: 0.1753
 - An autobiography of George Washington, the first president of the United States: -0.0321
 - Spider-Man is fighting against the Green Goblin in another universe: 0.3556
我想看一部超級英雄電影
 - A movie about a group of friends who go on a road trip: 0.2784
 - A romantic comedy about a couple who meet at a wedding: 0.1435
 - An autobiography of George Washington, the first president of the United States: 0.0242
 - Spider-Man is fighting against the Green Goblin in another universe: 0.3834


In [141]:
torch.topk(similarities[0], k=4)

torch.return_types.topk(
values=tensor([ 0.3556,  0.2611,  0.1753, -0.0321]),
indices=tensor([3, 0, 1, 2]))

In [13]:
for idx_i, sentence1 in enumerate(queries):
    print(sentence1)
    for idx_j in torch.topk(similarities[idx_i], k=4).indices:
        print(f" - {movies[idx_j]: <30}: {YELLOW}{similarities[idx_i][idx_j]:.4f}{END}")

I want to watch an exciting superhero movie
 - Spider-Man is fighting against the Green Goblin in another universe: 0.3556
 - A movie about a group of friends who go on a road trip: 0.2611
 - A romantic comedy about a couple who meet at a wedding: 0.1753
 - An autobiography of George Washington, the first president of the United States: -0.0321
我想看一部超級英雄電影
 - Spider-Man is fighting against the Green Goblin in another universe: 0.3834
 - A movie about a group of friends who go on a road trip: 0.2784
 - A romantic comedy about a couple who meet at a wedding: 0.1435
 - An autobiography of George Washington, the first president of the United States: 0.0242


## OpenAI

[Documentation](https://platform.openai.com/docs/guides/embeddings?lang=python)

![](https://i.redd.it/lpf0u9nbj7w41.jpg)

In [32]:
api_key = os.getenv("OPENAI_API_KEY")
if not api_key and userdata:
    # If running in Google Colab, try to get the API key from userdata
    api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable is not set")

client = AsyncOpenAI(api_key=api_key, max_retries=5)

### Compute embeddings

In [15]:
res = await client.embeddings.create(input=queries[0], model="text-embedding-3-small")
embedding = res.data[0].embedding
print(len(embedding))
print(embedding[:5])

1536
[-0.010025657713413239, 0.007190898526459932, -0.08766952902078629, 0.010150458663702011, -0.025221630930900574]


In [16]:
print(f"Total tokens: {res.usage.total_tokens}")

Total tokens: 8


In [17]:
# Convert to torch.Tensor for cosine similarity
_query_embeddings = await asyncio.gather(
    *[
        client.embeddings.create(input=query, model="text-embedding-3-small")
        for query in queries
    ]
)
query_embeddings = torch.Tensor(
    [embedding.data[0].embedding for embedding in _query_embeddings]
)
_movie_embeddings = await asyncio.gather(
    *[
        client.embeddings.create(input=movie, model="text-embedding-3-small")
        for movie in movies
    ]
)
movie_embeddings = torch.Tensor(
    [embedding.data[0].embedding for embedding in _movie_embeddings]
)

In [18]:
torch.Tensor(movie_embeddings)

tensor([[ 0.0125,  0.0218, -0.0479,  ...,  0.0026, -0.0386,  0.0079],
        [-0.0326,  0.0203, -0.0673,  ...,  0.0057, -0.0081, -0.0111],
        [ 0.0083, -0.0117,  0.0179,  ...,  0.0175, -0.0041, -0.0108],
        [-0.0362, -0.0462,  0.0150,  ..., -0.0224,  0.0127,  0.0131]])

### Computing similarity + retrieving top k

In [20]:
similarities = F.cosine_similarity(query_embeddings.unsqueeze(1), movie_embeddings.unsqueeze(0), dim=2)
similarities

tensor([[0.2881, 0.2119, 0.0501, 0.3344],
        [0.2241, 0.1764, 0.0754, 0.2568]])

In [21]:
YELLOW = "\033[33m"
END = "\033[0m"
for idx_i, sentence1 in enumerate(queries):
    print(sentence1)
    for idx_j in torch.topk(similarities[idx_i], k=4).indices:
        print(f" - {movies[idx_j]: <30}: {YELLOW}{similarities[idx_i][idx_j]:.4f}{END}")

I want to watch an exciting superhero movie
 - Spider-Man is fighting against the Green Goblin in another universe: 0.3344
 - A movie about a group of friends who go on a road trip: 0.2881
 - A romantic comedy about a couple who meet at a wedding: 0.2119
 - An autobiography of George Washington, the first president of the United States: 0.0501
我想看一部超級英雄電影
 - Spider-Man is fighting against the Green Goblin in another universe: 0.2568
 - A movie about a group of friends who go on a road trip: 0.2241
 - A romantic comedy about a couple who meet at a wedding: 0.1764
 - An autobiography of George Washington, the first president of the United States: 0.0754


### Calculate tokens and price

[OpenAI pricing](https://platform.openai.com/docs/pricing)

In [24]:
enc = tiktoken.encoding_for_model("text-embedding-3-small")

In [27]:
encoded = enc.encode(queries[0])
print(f"Total tokens: {len(encoded)}")
print(encoded)

Total tokens: 8
[40, 1390, 311, 3821, 459, 13548, 46244, 5818]


In [28]:
# price per 1M tokens
model_to_price = {
    "text-embedding-3-small": 0.02,
    "text-embedding-3-large": 0.13,
}


def get_token_count_and_price(
    texts: list[str], model: str = "text-embedding-3-small"
) -> tuple[int, float]:
    if model not in model_to_price:
        raise ValueError(f"Model {model} not supported")
    enc = tiktoken.encoding_for_model(model)
    token_count = sum(len(e) for e in enc.encode_batch(texts))
    price_per_1m_tokens = model_to_price[model]
    price = (token_count / 1_000_000) * price_per_1m_tokens
    return token_count, price

In [29]:
get_token_count_and_price(movies * 10, model="text-embedding-3-small")

(480, 9.600000000000001e-06)

### Handling rate limits

In [ ]:
# def log_backoff_attempt(retry_state: RetryCallState) -> None:
#     """
#     Logs a message before a retry attempt, detailing the attempt number,
#     the exception, and the wait time.
#     """
#     attempt_num = retry_state.attempt_number
#     exception = retry_state.outcome.exception() if retry_state.outcome else "N/A"
#     wait_time = retry_state.next_action.sleep if retry_state.next_action else 0.0
#     func_name = retry_state.fn.__name__ if retry_state.fn else "N/A"

#     logger.info(
#         f"Backing off for function '{func_name}': "
#         f"Attempt {attempt_num} failed due to '{exception.__class__.__name__}: {exception}'. "
#         f"Waiting {wait_time:.2f} seconds before next attempt."
#     )


# @retry(
#     wait=wait_random_exponential(min=1, max=60),
#     stop=stop_after_attempt(6),
#     before_sleep=log_backoff_attempt,
# )
# async def embedding_with_backoff(**kwargs):
#     return await client.embeddings.create(**kwargs)

# Setting up a vector database

[LanceDB documentation](https://lancedb.github.io/lancedb/basic/)

In [216]:
# We'll use the "overview" column as the text to embed
ds[0]

{'id': 5,
 'budget': 4000000,
 'genres': ['Crime', 'Comedy'],
 'keywords': ['hotel',
  "new year's eve",
  'witch',
  'bet',
  'hotel room',
  'sperm',
  'los angeles',
  'hoodlum',
  'woman director',
  'episode film'],
 'original_language': 'en',
 'original_title': 'Four Rooms',
 'overview': "It's Ted the Bellhop's first night on the job...and the hotel's very unusual guests are about to place him in some outrageous predicaments. It seems that this evening's room service is serving up one unbelievable happening after another.",
 'popularity': 22.87623,
 'production_companies': ['Miramax Films', 'A Band Apart'],
 'production_countries': ['United States of America'],
 'release_date': datetime.date(1995, 12, 9),
 'revenue': 4300000,
 'runtime': 98.0,
 'spoken_languages': ['English'],
 'status': 'Released',
 'tagline': "Twelve outrageous guests. Four scandalous requests. And one lone bellhop, in his first day on the job, who's in for the wildest New year's Eve of his life.",
 'title': 'F

In [217]:
overviews = ds["overview"]
print(len(overviews))
overviews[:5]

4800


["It's Ted the Bellhop's first night on the job...and the hotel's very unusual guests are about to place him in some outrageous predicaments. It seems that this evening's room service is serving up one unbelievable happening after another.",
 'Princess Leia is captured and held hostage by the evil Imperial forces in their effort to take over the galactic Empire. Venturesome Luke Skywalker and dashing captain Han Solo team together with the loveable robot duo R2-D2 and C-3PO to rescue the beautiful princess and restore peace and justice in the Empire.',
 "Nemo, an adventurous young clownfish, is unexpectedly taken from his Great Barrier Reef home to a dentist's office aquarium. It's up to his worrisome father Marlin and a friendly but forgetful fish Dory to bring Nemo home -- meeting vegetarian sharks, surfer dude turtles, hypnotic jellyfish, hungry seagulls, and more along the way.",
 "A man with a low IQ has accomplished great things in his life and been present during significant his

## sentence-transformers

In [220]:
overview_embeddings = model.encode(overviews)
overview_embeddings[0][:5]

array([ 0.0797189 , -0.22588545, -0.08258644, -0.06734926,  0.07945839],
      dtype=float32)

In [227]:
df = ds.to_pandas()
df["vector"] = overview_embeddings.tolist()
df.iloc[0]

id                                                                      5
budget                                                            4000000
genres                                                    [Crime, Comedy]
keywords                [hotel, new year's eve, witch, bet, hotel room...
original_language                                                      en
original_title                                                 Four Rooms
overview                It's Ted the Bellhop's first night on the job....
popularity                                                       22.87623
production_companies                        [Miramax Films, A Band Apart]
production_countries                           [United States of America]
release_date                                                   1995-12-09
revenue                                                           4300000
runtime                                                              98.0
spoken_languages                      

In [208]:
db = lancedb.connect("../data/tmdb-5000-movies-sentence-transformers")

In [228]:
tbl = db.create_table("movies", data=df)

## OpenAI

In [30]:
overviews = ds["overview"]
get_token_count_and_price(overviews, model="text-embedding-3-small")

(311562, 0.00623124)

In [ ]:
_overview_embeddings = await asyncio.gather(
    *[
        # embedding_with_backoff(input=overview, model="text-embedding-3-small")
        client.embeddings.create(input=overview, model="text-embedding-3-small")
        for overview in overviews
    ]
)
overview_embeddings = [
    embedding.data[0].embedding for embedding in _overview_embeddings
]
overview_embeddings[0][:5]

In [ ]:
df = ds.to_pandas()
df["vector"] = overview_embeddings

In [ ]:
db = lancedb.connect("../data/tmdb-5000-movies-openai")
tbl = db.create_table("movies", data=df)